In [1]:

import os

In [2]:

%pwd

'f:\\AI Projects\\End-to-End-NLP-Project-Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'f:\\AI Projects\\End-to-End-NLP-Project-Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name
           
        )

        return model_evaluation_config

In [10]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import evaluate
import torch
import pandas as pd
from tqdm import tqdm


In [17]:
class ModelEvaluation:
    def __init__(self, config):
        self.config = config

    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i: i + batch_size]

    def calculate_metric_on_test_ds(
        self,
        dataset,
        metric,
        model,
        tokenizer,
        batch_size=16,
        device="cuda" if torch.cuda.is_available() else "cpu",
        column_text="dialogue",
        column_summary="summary",
    ):

        article_batches = list(
            self.generate_batch_sized_chunks(dataset[column_text], batch_size)
        )

        target_batches = list(
            self.generate_batch_sized_chunks(dataset[column_summary], batch_size)
        )

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches),
            total=len(article_batches)
        ):

            # T5 requires prefix
            article_batch = ["summarize: " + article for article in article_batch]

            inputs = tokenizer(
                article_batch,
                max_length=512,
                truncation=True,
                padding=True,
                return_tensors="pt"
            )

            summaries = model.generate(
                input_ids=inputs["input_ids"].to(device),
                attention_mask=inputs["attention_mask"].to(device),
                num_beams=4,
                length_penalty=1.0,
                max_length=128,
            )

            decoded_summaries = [
                tokenizer.decode(
                    s,
                    skip_special_tokens=True,
                    clean_up_tokenization_spaces=True
                )
                for s in summaries
            ]

            metric.add_batch(
                predictions=decoded_summaries,
                references=target_batch
            )

        score = metric.compute()
        return score

    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"

        # Load fine-tuned T5 model (LOCAL)
        tokenizer = AutoTokenizer.from_pretrained(
            self.config.tokenizer_path,
            local_files_only=True
        )

        model = AutoModelForSeq2SeqLM.from_pretrained(
            self.config.model_path,
            local_files_only=True
        ).to(device)

        # Load dataset
        dataset = load_from_disk(self.config.data_path)

        # Load ROUGE metric
        rouge_metric = evaluate.load("rouge")

        score = self.calculate_metric_on_test_ds(
            dataset["test"][0:10],
            rouge_metric,
            model,
            tokenizer,
            batch_size=2,
            column_text="dialogue",
            column_summary="summary",
        )

        # Ensure directory exists
        os.makedirs(self.config.root_dir, exist_ok=True)

        # Save results
        df = pd.DataFrame([score])
        df.to_csv(self.config.metric_file_name, index=False)

        print("Evaluation Completed")
        print(score)

In [18]:
try:
    config_manager = ConfigurationManager()
    evaluation_config = config_manager.get_model_evaluation_config()
    model_evaluator = ModelEvaluation(config=evaluation_config)
    model_evaluator.evaluate()
except Exception as e:
    raise e

[2026-02-13 15:59:56,404: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-02-13 15:59:56,410: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-13 15:59:56,412: INFO: common: created directory at: artifacts]
[2026-02-13 15:59:56,415: INFO: common: created directory at: artifacts/model_evaluation]


100%|██████████| 5/5 [00:47<00:00,  9.51s/it]

[2026-02-13 16:00:59,385: INFO: rouge_scorer: Using default tokenizer.]


Evaluation Completed
{'rouge1': np.float64(0.22053466000473929), 'rouge2': np.float64(0.06418107647001686), 'rougeL': np.float64(0.1895456290084985), 'rougeLsum': np.float64(0.19030160818913522)}
